In [ ]:
import csv
import tempfile
import psycopg2 
from zipfile import ZipFile

def save_new_raw_data():
    local_path = 'melbourne-housing-snapshot.zip'

    with tempfile.TemporaryDirectory() as dirpath:
        with ZipFile(local_path, "r") as zipfile:
            names_list = zipfile.namelist()

            if names_list: 
                csv_file_path = zipfile.extract(names_list[0], path=dirpath)

                with open(csv_file_path, mode="r", encoding="utf-8") as csv_file:
                    reader = csv.DictReader(csv_file)

                    row = next(reader, None)
                    data= []
                    for i, row in enumerate(reader):
                        
                        print(i, row)
                        data.append(row)
    return data

def load(data) -> None:
    conn = psycopg2.connect(user="postgres",
                                password="Salsha12",
                                host="localhost",
                                port="5432",
                                database="Coba coba")
    cur = conn.cursor()

    cur.execute(''' DROP TABLE IF EXISTS melbourne ''')
 
    create_script= (
        """
        CREATE TABLE IF NOT EXISTS melbourne  (
            Postcode SERIAL NOT NULL,
            Address VARCHAR (55) NOT NULL,
            Rooms INT NOT NULL,
            Type VARCHAR (5) NOT NULL,
            Price INT NOT NULL,
            Suburb VARCHAR (55) NOT NULL,
            Method VARCHAR (5) NOT NULL,
            SellerG VARCHAR (55) NOT NULL,
            Date DATE NOT NULL,
            Distance INT NOT NULL,
            Bedroom2 INT NOT NULL,
            Bathroom INT NOT NULL,
            Car INT NOT NULL,
            Landsize INT NOT NULL,
            BuildingArea INT NOT NULL,
            YearBuilt INT NOT NULL,
            CouncilArea VARCHAR (55) NOT NULL,
            Lattitude VARCHAR (55) NOT NULL,
            Longtitude VARCHAR (55) NOT NULL,
            Regionname VARCHAR (55) NOT NULL,
            Propertycount INT NOT NULL
        );
        """
    )
    cur.execute(create_script)
    
    class NaN:
        def __init__(self, default=None):
          self.val = default
        def __repr__(self):
          return 'NaN' 
    for row in data:
        row= {a:0 if not b else b for a,b in row.items()}
        row['SellerG']= row['SellerG'].replace("'", '')
        cur.execute(
                f"""
                INSERT INTO melbourne (Suburb, Address, Rooms, Type, Price, Method, SellerG, Date, Distance, Postcode, Bedroom2, Bathroom, Car, Landsize, BuildingArea,YearBuilt, CouncilArea, Lattitude, Longtitude, Regionname, Propertycount)
                VALUES ('{row['Suburb']}','{row['Address']}', {row['Rooms']}, '{row['Type']}', {row['Price']}, '{row['Method']}', '{row['SellerG']}', TO_DATE('{row['Date']}', 'DD-MM-YYYY'), {row['Distance']}, {row['Postcode']}, {row['Bedroom2']}, {row['Bathroom']}, {row['Car']}, {row['Landsize']}, {row['BuildingArea']}, {row['YearBuilt']}, '{row['CouncilArea']}', '{row['Lattitude']}', '{row['Longtitude']}', '{row['Regionname']}', {row['Propertycount']}); 
                """
            )
        
    conn.commit()
    conn.close()
    print('Process Completed')

if __name__ == "__main__":
    load(save_new_raw_data())